## Functionality 4

* *Hypothesis* in which we are working are the following: undirect graph retrived filtering the intervals of times that has as weight the sum of all the weight of the edges between the two original nodes. 

* *Costraint*:  create two partitions A and B of nodes such that user1 belongs to A and user2 belongs to B 

* *Goal*: minimizing the weight of the cut 

PIPELINE:

1. converting the interval of time
2. filtering dictionary using the interval of time
3. aggregate the edges between two nodes summing their weights
* aggregate by user2 for every first user
* create a dictionary with keys= (user1,user2) and weight the sum of the weights between them
4. defining Stoer Wagner algorithm for two users
* define supernode
* define step
* define iteration
* use all these functions to create Stoer Wagner
5. apply the algorithm to the dictionary to retrieve the number of cuts ans their total weight


In [141]:
import pickle

with open('../data/interactions.pickle', 'rb') as handle:
    my_dict = pickle.load(handle)

#(user2,time_stamp,score_associated_to_interaction)

### Part1: converting the interval of time

In [1]:
def from_date_to_int(date):
    '''
    INPUT: (day,month,year)
    OUTPUT: correspondent integer
    '''
    if date[1]==2:
        integer=date[0]+date[1]*28+(date[2]-1970)*365
    if date[1] in [4,6,9,11]:
        integer=date[0]+date[1]*30+(date[2]-1970)*365
    else:
        integer=date[0]+date[1]*31+(date[2]-1970)*365
    return(integer)

In [2]:
def convert_interval(interval):
    '''
    INPUT= interval of time in format [(dd,mm,yyyy),(dd,mm,yyyy),(dd,mm,yyyy),(dd,mm,yyyy)] --> [start1,end1,start2,end2]
    OUTPUT: interval of time in format [encoded_start1,encoded_end1,encoded_start2,encoded_end2] where encoded are integers values
    '''
    new_int=[]
    for date in interval:
        new_int.append(from_date_to_int(date))
    
    return( new_int)

In [3]:
#just to try
#from_date_to_int((22,5,1973))

In [142]:
#imagine to take in input date in format [(dd,mm,yyyy),(dd,mm,yyyy)]

#first day included, second day escluded
interval=[(29,7,2008),(30,7,2008)] 

converted_interval=convert_interval(interval)
converted_interval


[14116, 14117]

### Part2: filtering dictionary using the interval

In [143]:
prova={}
prova[1]=[(2,736383,10),(3,43,5)]
prova[2]=[(3,729305,2)]

In [144]:
def filter_dictionary(dictionary,converted_interval): 
    filtered_dictionary={}
    for key,value in dictionary.items():
        for elem in value:
            if ( (elem[1]>=converted_interval[0] and elem[1]<converted_interval[1]) ): # or (elem[1]>converted_interval[2] and elem[1]<converted_interval[3]) ):
                try:
                    filtered_dictionary[key].append(elem)
                except:
                    filtered_dictionary[key]=[elem]
    return(filtered_dictionary)  

In [145]:
filtered_dictionary=filter_dictionary(my_dict,converted_interval)


In [146]:
len(filtered_dictionary)

232

In [148]:
if (len(filtered_dictionary)==0):
    print('no elements in this interval')

### Part 3: aggregate the edges between two nodes summing their weights

firstly let's reduce the value of every user by summing they score if the other user is the same:
1:[(2,3343,10),(2,2434,5),(78,433,5)] becames 1:{2:15, 78:5}]

In [149]:
prova={}
prova[1]=[(2,1111,10),(2,11111,5)]
prova[2]=[(1,1111,5),(2,34343,10)]

 I group for every user all the relations that he had with another user--> rough way

now we need to costruct  just one edge between the two nodes

In [150]:
intermediate={}
for key,value in filtered_dictionary.items():
    dizio={}
    for elem in value:
        #check user2 in keys: if it is NOT in the dictionary
        try:
            previous=dizio[elem[0]]
            dizio[elem[0]]=previous+elem[2]
        except:
            dizio[ elem[0] ]=elem[2]
    
    intermediate[key]=dizio
        




In [151]:
len(intermediate)

232

In [152]:
#intermediate

merge all the edges

In [64]:
'''
prova={}
prova[1]={2:10,3:5}
prova[2]={1:5,3:10}
prova[3]={2:5,1:10,8:2}
type(prova[1])
len(prova)
''';

In [153]:
final_dict={}
for key,value in intermediate.items():

    for key1,value1 in value.items():
    
        if ((str(key)+','+str(key1)) in final_dict):
            previous=final_dict[str(key)+','+str(key1)]
            final_dict[str(key)+','+str(key1)] =previous+value1

        if ((str(key1)+','+str(key)) in final_dict):
            previous=final_dict[str(key1)+','+str(key)]
            final_dict[str(key1)+','+str(key)]=previous+value1

        else:
            final_dict[str(key)+','+str(key1)]=value1


        

In [154]:
len(final_dict)

439

In [156]:
#final_dict

final_dict is a dictionary in which we have all the edges of our indirect graph in the format 'user1,user2' : value

# PART 4

#### Find MIN CUT in undirect graph

I need a function to merge two nodes and that give the new graph encoded as dictionary as output 


In [159]:
def supernode(G, node1 ,node2):
       '''
       INPUT: G=graph represented as dictionary
              node1
              node2
       OUTPUT: new graph in which node1 and node2 have been merged together in which the new edge will have the notation ['node1|node2,user']=value
              new_edges
       '''
       
       #find edges to remove
       old_edges=[]
       for k,v in G.items():

              if ( (k.split(',')[-1])==node1 or (k.split(',')[-2])==node1 ):
                     old_edges.append(k)
                     
              if ( (k.split(',')[-1])==node2 or (k.split(',')[-2])==node2 ):
                     old_edges.append(k)

       #remove doubles from old_edges 
       old_edges=list(set(old_edges))
       

       #new_graph
       new_dict=G.copy()
       for key in old_edges:
              del new_dict[key]

       #remove edge between node1 and node2
       for old_edge in old_edges:
              if (old_edge.split(',')[-1]==node1 and old_edge.split(',')[-2]==node2):
                     old_edges.remove(old_edge)
              if (old_edge.split(',')[-2]==node1 and old_edge.split(',')[-1]==node2):
                     old_edges.remove(old_edge)      
       
       
       #now i need to find all the edges that go to the same nodes and sum them to produce a new edge that will have the following shape {node1,node2,user: sum edges from node1 to user + sum edges node2 a user)}
       new_edges={}
       for edge in old_edges:
              

              if (edge.split(',')[-1]==str(node1) or edge.split(',')[-1]==str(node2)):
                     
                     try:
                            previous=new_edges[str(node1)+'|'+str(node2)+','+edge.split(',')[-2]]
                            new_edges[str(node1)+'|'+str(node2)+','+edge.split(',')[-2]]=previous+G[edge]
                     except:
                            new_edges[str(node1)+'|'+str(node2)+','+edge.split(',')[-2]]=G[edge]
                            
              if (edge.split(',')[-2]==str(node1) or edge.split(',')[-2]==str(node2)):
                     
                     try:
                            previous=new_edges[str(node1)+'|'+str(node2)+','+edge.split(',')[-1]]
                            new_edges[str(node1)+'|'+str(node2)+','+edge.split(',')[-1]]=previous+G[edge]
                     except:
                            new_edges[str(node1)+'|'+str(node2)+','+edge.split(',')[-1]]=G[edge]

       
       for k,v in new_edges.items():
              new_dict[k]=v
       

       return new_dict,new_edges

In [160]:
'''
nuovo_diz,new=supernode(prova,'4','5')
nuovo_diz
nuovo_diz,new=supernode(nuovo_diz,'4|5','3')
nuovo_diz
''';


Now I need to define a function that compute the elementary step given a starting point: i retrieve the edge with the max weight between the edges attached to the starting point and then i create a supernode between the starting point and the node connect to this edge that I have found


I need to define the simple step to reduce every subgrap


In [161]:
def step(G,s,t):
    '''
    INPUT: graph G, node s (source node), node t (sink node) [  format 's' and 't']
    OUTPUT: new graph given merging sorce node with the most tightly connected vertex
    '''
    
    #search the edges connected to s
    
    other_user=[]
    value=0
    for k,v in G.items():
        
        if(k.split(',')[-2]==s):
            #not the sink
            if(k.split(',')[-1]!=t):
                if v>value:
                    value=v
                    other_user.append(k.split(',')[-1])

        if (k.split(',')[-1]==s):
            if(k.split(',')[-2]!=t):
                if v>value:
                    value=v
                    other_user.append(k.split(',')[-2])

    
    # I found the edge that has the biggest weight 
    subgraph,new_edges = supernode(G,s,other_user[-1])

    return subgraph,new_edges

In [162]:
'''  
prova={}
prova['1,2']=5
prova['1,4']=2
prova['2,4']=1
prova['2,3']=6
prova['3,4']=1
prova['3,5']=3
prova['4,5']=2

ris1,inutile=step(prova,'1','5')
ris2,inutile=step(ris1,'1|2','5')
ris3,inutile=step(ris2,'1|2|3','5')
ris3
''';

Now i can implement an ITERATION of Stoer Wagner

In [163]:
def iteration(G,s,t):
    '''
    INPUT: grapg G, source node s and sink node t
    OUTPUT: the values of the cut and the last two elements of the graph that will be merged before the next iteration
    '''
    #initialization
    iteration_dict,new_edge= step(G,s,t)
   
    while ( len(iteration_dict)>2  and list(new_edge)[0].split(',')[1]!=t) :
        
        start=list(new_edge)[0].split(',')[0]
        iteration_dict,new_edge= step(iteration_dict,start,t)

    cut=list(iteration_dict.values())[0]
    #node to merge:
    node1_to_merge=list(iteration_dict.keys())[0].split(',')[0].split('|')[-1]
    node2_to_merge=list(iteration_dict.keys())[0].split(',')[-1]
    return cut,node1_to_merge,node2_to_merge

In [10]:
# faccio una prova
'''
cuts=[]
s='1'
t='5'
iteration_graph=prova.copy()
cut,node1_to_merge,node2_to_merge=iteration(iteration_graph,s,t)
cuts.append(cut)

#finito la prima iterazione
# creo il nuovo grafo e parto da li
prova,new_e=supernode(prova,node1_to_merge,node2_to_merge)
iteration_graph=prova.copy()
t=node1_to_merge+'|'+node2_to_merge
cut,node1_to_merge,node2_to_merge=iteration(iteration_graph,s,t)

#finita altra iterazione che va bene
#credo nuovo grafo e riparto da li
prova,new_e=supernode(prova,node1_to_merge,node2_to_merge)
iteration_graph=prova.copy()
t=node1_to_merge+'|'+node2_to_merge
cut,node1_to_merge,node2_to_merge=iteration(iteration_graph,s,t)

# anche questa iterazione è andata bene: devo fare l'ultima
prova,new_e=supernode(prova,node1_to_merge,node2_to_merge)

# anche questa iterazione è andata bene: devo fare l'ultima
prova,new_e=supernode(prova,node1_to_merge,node2_to_merge)
''';

-------

In [164]:
def Stoer_Wagner(G,s,t):

    cuts=[]
    #initialize
    iteration_graph=G.copy()
    cut,node1_to_merge,node2_to_merge=iteration(iteration_graph,s,t)
    cuts.append(cut)
    #smaller_graph
    G,new_e=supernode(G,node1_to_merge,node2_to_merge)
    t=node1_to_merge+'|'+node2_to_merge

    while len(G)>2:

        iteration_graph=G.copy()
        t=node1_to_merge+'|'+node2_to_merge
        cut,node1_to_merge,node2_to_merge=iteration(iteration_graph,s,t)
        cuts.append(cut)
        G,new_e=supernode(G,node1_to_merge,node2_to_merge)

    if len(G)==1:
        cuts.append(list(G.values())[0])

    return(min(cuts))

In [165]:
prova={}
prova['1,2']=3
prova['1,4']=4
prova['2,4']=1
prova['2,3']=6
prova['3,4']=1
prova['3,5']=2
prova['4,5']=3
prova['5,6']=2



In [166]:
dizionario=prova
#dizionario=final_dict
taglio=Stoer_Wagner(dizionario,'1','7')

In [167]:
taglio

2